<a href="https://colab.research.google.com/github/garfield-gray/Optimization/blob/main/Convex/IntProg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
# phase 1 (2 rules!!)
# write standardizing:) in a function if you could!!!
# And that's it:)


# IntProg

In [46]:
from scipy.optimize import LinearConstraint, milp
import numpy as np

In [47]:
c = np.array([3, 4])
A = np.array([[-3, -1], [-1, -2]])
b_u = np.array([-4, -4])
b_l = np.full_like(b_u, -np.inf, dtype=float)
# b_l = np.full_like(b_u, 0, dtype=float)

In [48]:
constraints = LinearConstraint(A, b_l, b_u)

In [49]:
integrality = np.ones_like(c)
res = milp(c=c, constraints=constraints, integrality=integrality)
res.x

array([2., 1.])

In [50]:
res = milp(c=c, constraints=constraints)  # OR:
# from scipy.optimize import linprog; res = linprog(c, A, b_u)
res.x

array([0.8, 1.6])

# Linprog

In [51]:
from scipy.optimize import linprog
c = [3, 4]
A = [[-3, -1], [-1, -2]]
b = [-4, -4]
x0_bounds = (0, None)
x1_bounds = (0, None)
res = linprog(c, A_ub=A, b_ub=b, bounds=[x0_bounds, x1_bounds])
res.fun


8.8

In [52]:
res.x

array([0.8, 1.6])

In [53]:
res.message

'Optimization terminated successfully. (HiGHS Status 7: Optimal)'

# The Algorithm

In [92]:
def Standardize (tableau):
    """Attention!! the bfs must already exist!!"""
    m, n = tableau.shape
    m -=1
    n -=2

    for i in range(m):

        tableau[0] -= tableau[0, int(tableau[i+1,0]+1)] * tableau[i+1]

    tableau[0, 0] = 1

    return tableau

In [93]:
t = np.array([[ 1, -1,  4,  0,  0,  0, -1, -1, -1,  0],
              [ 5,  1, -1, -1,  0,  0,  1,  0,  0,  1],
              [ 6, -1,  4,  0,  1,  0,  0,  1,  0,  7],
              [ 7,  3,  1,  0,  0,  1,  0,  0,  1, 18]])
Standardize(t)

array([[ 1,  2,  8, -1,  1,  1,  0,  0,  0, 26],
       [ 5,  1, -1, -1,  0,  0,  1,  0,  0,  1],
       [ 6, -1,  4,  0,  1,  0,  0,  1,  0,  7],
       [ 7,  3,  1,  0,  0,  1,  0,  0,  1, 18]])

In [55]:
def CoreSimplex(tableau):
    basics = tableau[1:,0].copy() # start drom zero as the first element!!
    m, n = tableau.shape
    m -=1
    n -=2
    while True:
        # Check if we have an optimal solution (all entries in the objective row are negative)
        if np.all(tableau[0, 1:-1] <= 0):
            break
        # Pivot column (most negative entry in the objective row)
        pivot_col = np.argmax(tableau[0, 1:-1])+1
        # Pivot row
        ratios = np.divide(tableau[1:, -1], tableau[1:, pivot_col])
        # print(ratios)
        valid_ratios = ratios[tableau[1:, pivot_col] > 0]
        # print(valid_ratios)
        # Check if there's no valid ratio which means unbounded
        if len(valid_ratios) == 0:
            raise ValueError("The problem is unbounded.")
        ########could be written better################
        pivot_row = np.where(ratios == valid_ratios.min())[0][0] +1
        # Pivot element
        pivot_element = tableau[pivot_row, pivot_col]
        # Update the pivot row
        tableau[pivot_row] /= pivot_element
        # Update the other rows
        for i in range(m + 1):
            if i != pivot_row:
                # print(i)
                tableau[i] -= tableau[i, pivot_col] * tableau[pivot_row]

        basics[pivot_row-1] = pivot_col-1
    tableau[1:, 0] = basics
    tableau[0, 0] = 1
    return tableau


In [56]:
t = np.array(
    [[ 1, -1,  4,  0,  0,  0,  0],
     [ 2, -1,  1,  1,  0,  0,  1],
     [ 3, -1,  4,  0,  1,  0,  7],
     [ 4,  3,  1,  0,  0,1.0, 18]])
print(np.round(CoreSimplex(t), 4))

[[ 1.      0.      0.      0.     -1.      0.     -7.    ]
 [ 1.      0.      1.     -0.3333  0.3333  0.      2.    ]
 [ 0.      1.      0.     -1.3333  0.3333  0.      1.    ]
 [ 4.      0.      0.      4.3333 -1.3333  1.     13.    ]]


In [104]:
def Phase1(tableau):
    """takes the tableau with no feasible solution and outpus with bfs (not standardize)"""
    basics = tableau[1:,0].copy() # start drom zero as the first element!!
    m, n = tableau.shape
    m -=1
    n -=2
    b = tableau[1:, -1]
    p = np.eye(m+1)
    p[np.where(b<0)+np.ones_like(np.where(b<0))] *= -1
    Rtableau = p @ tableau
    Rtableau = np.insert(Rtableau, -1, np.vstack((-np.ones(m),np.eye(m))).T, axis=1)
    Rtableau[0, 0] = 1
    # fixing the basic variables
    Rtableau[1:, 0] = np.arange(n, n+m)
    Rtableau = Standardize(Rtableau)
    print(Rtableau)
    Rtableau = CoreSimplex(Rtableau)

    print(np.round(Rtableau,2))









    #####################to fix
    tableau = np.delete(Rtableau, np.s_[n:n+m], axis=1)


    return 0
    while True:
        # Check if we have an optimal solution (all entries in the objective row are negative)
        if np.all(tableau[0, 1:-1] <= 0):
            break
        # Pivot column (most negative entry in the objective row)
        pivot_col = np.argmax(tableau[0, 1:-1])+1
        # Pivot row
        ratios = np.divide(tableau[1:, -1], tableau[1:, pivot_col])
        # print(ratios)
        valid_ratios = ratios[tableau[1:, pivot_col] > 0]
        # print(valid_ratios)
        # Check if there's no valid ratio which means unbounded
        if len(valid_ratios) == 0:
            raise ValueError("The problem is unbounded.")
        ########could be written better################
        pivot_row = np.where(ratios == valid_ratios.min())[0][0] +1
        # Pivot element
        pivot_element = tableau[pivot_row, pivot_col]
        # Update the pivot row
        tableau[pivot_row] /= pivot_element
        # Update the other rows
        for i in range(m + 1):
            if i != pivot_row:
                # print(i)
                tableau[i] -= tableau[i, pivot_col] * tableau[pivot_row]

        basics[pivot_row-1] = pivot_col-1
    tableau[1:, 0] = basics
    tableau[0, 0] = 1
    return tableau


In [105]:
t = np.array(
    [[ 1, -1,  4,  0,  0,  0,  0],
     [ 2, -1,  1,  1,  0,  0, -1],
     [ 3, -1,  4,  0,  1,  0,  7],
     [ 4,  3,  1,  0,  0,1.0, 18]])
np.round(Phase1(t), 4)

[[ 1.  2.  8. -1.  1.  1.  0.  0.  0. 26.]
 [ 5.  1. -1. -1.  0.  0.  1.  0.  0.  1.]
 [ 6. -1.  4.  0.  1.  0.  0.  1.  0.  7.]
 [ 7.  3.  1.  0.  0.  1.  0.  0.  1. 18.]]
[[ 1.    0.    0.    0.   -1.    0.   -1.   -2.   -1.   -7.  ]
 [ 0.    1.    0.    0.   -0.08  0.31  0.   -0.08  0.31  5.  ]
 [ 1.    0.    1.    0.    0.23  0.08  0.    0.23  0.08  3.  ]
 [ 2.    0.    0.    1.   -0.31  0.23 -1.   -0.31  0.23  1.  ]]


0

In [78]:
np.arange(5, 5+3)

array([5, 6, 7])

In [ ]:

# needs to checked if variables are all positive
def simplex(c, A, b):
    # Number of variables
    n = len(c)
    # Number of constraints
    m = len(b)
    # creation of bfs(Basic feasible solution through Artificial variables)
    Rtableau = np.zeros((m + 1, n + m + 1))
    Rtableau[1:, :n] = A
    Rtableau[1:, -1] = b
    p = np.eye(m+1)
    p[np.where(b<0)+np.ones_like(np.where(b<0))] *= -1
    Rtableau = p @ Rtableau
    Rtableau[1:, n:n+m] = np.eye(m)
    #
    Rtableau[0, n:n+m] = -np.ones(m)
    # Standardizing:)
    for i in range(m):
        Rtableau[0] += Rtableau[i+1]


    Rtableau = CoreSimplex(Rtableau)
    # checking the feasibility
    # if
    tableau = np.delete(Rtableau, np.s_[n:n+m], axis=1)


    tableau[0, :-1] = -c
    # Standardizing :)
    for i in range(m):
        # print(i)
        # print((tableau[0, np.where(tableau[i+1]==1)[0]]))
        tableau[0] -= tableau[0, np.where(tableau[i+1]==1)[0]] * tableau[i+1]
    # print(tableau)
    tableau = CoreSimplex(tableau)


    return tableau


In [ ]:
A = np.array([
    [3, 1,-1, 0],
    [1, 2, 0,-1],

])
b = np.array([4, 4])
c = np.array([3, 4, 0, 0])
np.round(simplex(c, A, b),4)

array([[ 0. ,  0. , -0.4, -1.8,  8.8],
       [ 1. ,  0. , -0.4,  0.2,  0.8],
       [ 0. ,  1. ,  0.2, -0.6,  1.6]])

In [ ]:
# needs to checked if variables are all positive
def intSimplex(c, A, b):
    # Number of variables
    n = len(c)
    # Number of constraints
    m = len(b)
    # creation of bfs(Basic feasible solution through Artificial variables)
    Rtableau = np.zeros((m + 1, n + m + 1))
    Rtableau[1:, :n] = A
    Rtableau[1:, -1] = b
    p = np.eye(m+1)
    p[np.where(b<0)+np.ones_like(np.where(b<0))] *= -1
    Rtableau = p @ Rtableau
    Rtableau[1:, n:n+m] = np.eye(m)
    #
    Rtableau[0, n:n+m] = -np.ones(m)
    for i in range(m):
        Rtableau[0] += Rtableau[i+1]


    Rtableau = CoreSimplex(Rtableau)
    # checking the feasibility
    # if
    tableau = np.delete(Rtableau, np.s_[n:n+m], axis=1)


    tableau[0, :-1] = -c
    for i in range(m):
        # print(i)
        # print((tableau[0, np.where(tableau[i+1]==1)[0]]))
        tableau[0] -= tableau[0, np.where(tableau[i+1]==1)[0]] * tableau[i+1]
    # print(tableau)
    tableau = CoreSimplex(tableau)

    while True:
        # Check if we have an optimal solution (all entries in the objective row are negative)
        if np.all(np.modf(tableau[1:, -1])[0] == 0):
            break
        ########could be written better################
        # fracIdx = (np.where(np.modf(tableau[1:, -1])[0] != 0)[0]+1)[0]
        fracIdx = 2
        # print(tableau[fracIdx]-np.floor(tableau[fracIdx]))
        tableau = np.vstack((tableau, -(tableau[fracIdx]-np.floor(tableau[fracIdx]))))
        newVar = np.zeros(tableau.shape[0])
        newVar[-1] = 1
        tableau = np.insert(tableau, -1, newVar, axis=1)

        # tableau = simplex(tableau[0, :-1], tableau[1:, :-1], tableau[1:, -1])

        print(tableau)
        print(tableau[0, :-1])
        print( tableau[1:, :-1])
        print( tableau[1:, -1])

        break
    return tableau


In [ ]:
A = np.array([
    [3, 1,-1, 0],
    [1, 2, 0,-1],

])
b = np.array([4, 4])
c = np.array([3, 4, 0, 0])
np.round(intSimplex(c, A, b),4)

[[ 0.   0.  -0.4 -1.8  0.   8.8]
 [ 1.   0.  -0.4  0.2  0.   0.8]
 [ 0.   1.   0.2 -0.6  0.   1.6]
 [-0.  -0.  -0.2 -0.4  1.  -0.6]]
[ 0.   0.  -0.4 -1.8  0. ]
[[ 1.   0.  -0.4  0.2  0. ]
 [ 0.   1.   0.2 -0.6  0. ]
 [-0.  -0.  -0.2 -0.4  1. ]]
[ 0.8  1.6 -0.6]


array([[ 0. ,  0. , -0.4, -1.8,  0. ,  8.8],
       [ 1. ,  0. , -0.4,  0.2,  0. ,  0.8],
       [ 0. ,  1. ,  0.2, -0.6,  0. ,  1.6],
       [-0. , -0. , -0.2, -0.4,  1. , -0.6]])

In [67]:
import numpy as np

# Example 5x7 array
S = np.array([[1, 2, 3, 4, 5, 6, 7],
              [8, 9, 10, 11, 12, 13, 14],
              [15, 16, 17, 18, 19, 20, 21]])

# New column to insert
new_column = np.vstack((-np.ones(2),np.eye(2)))
# new_column = np.random.rand(2,3)
print(new_column)
# Print the original array
print("Original array:\n", S)

# Insert the new column at the 5th position (index 4)
S_modified = np.insert(S, -1, new_column.T, axis=1)

# Print the modified array
print("Modified array (with new column at 5th position):\n", S_modified)


[[-1. -1.]
 [ 1.  0.]
 [ 0.  1.]]
Original array:
 [[ 1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14]
 [15 16 17 18 19 20 21]]
Modified array (with new column at 5th position):
 [[ 1  2  3  4  5  6 -1 -1  7]
 [ 8  9 10 11 12 13  1  0 14]
 [15 16 17 18 19 20  0  1 21]]


In [ ]:
import numpy as np

# Example array
arr = np.array([-2.5, 2.5, 3.0, 4.2, -3.7])

# Get the integral part using floor
integral_part = np.floor(arr)

# Get the fractional part by subtracting the integral part from the original array
fractional_part = arr - integral_part

print("Array:", arr)
print("Integral part:", integral_part)
print("Fractional part:", fractional_part)


Array: [-2.5  2.5  3.   4.2 -3.7]
Integral part: [-3.  2.  3.  4. -4.]
Fractional part: [0.5 0.5 0.  0.2 0.3]


In [ ]:
newVar = np.zeros(5)
newVar[-1] = 1
newVar[1:-1]

array([0., 0., 0.])